In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn import metrics

In [138]:
data = pd.read_csv("ElecDeviceRatingPrediction_Milestone2.csv")

In [139]:
data.head()

,brand,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,graphic_card_gb,weight,warranty,Touchscreen,msoffice,Price,rating,Number of Ratings,Number of Reviews
0,ASUS,Intel,Pentium Quad,Not Available,4 GB,LPDDR4X,0 GB,512 GB,64-bit Windows,0 GB,Casual,No warranty,No,No,45990,Bad Rating,0,0
1,ASUS,Intel,Core i5,10th,8 GB,LPDDR4,0 GB,512 GB,32-bit DOS,4 GB,Casual,No warranty,No,No,106167,Good Rating,1881,241
2,ASUS,Intel,Celeron Dual,Not Available,4 GB,DDR4,0 GB,512 GB,64-bit Windows,0 GB,Casual,No warranty,No,No,22990,Bad Rating,0,0
3,Lenovo,Intel,Core i3,11th,4 GB,DDR4,256 GB,0 GB,64-bit Windows,0 GB,Casual,1 year,No,Yes,44990,Good Rating,42,4
4,HP,AMD,Ryzen 7,Not Available,16 GB,DDR4,512 GB,0 GB,64-bit Windows,2 GB,Casual,1 year,No,Yes,79990,Bad Rating,23,6


In [140]:
mode_value = data['processor_gnrtn'].mode()[0]

# Replace 'not available' with the mode
data.loc[data['processor_gnrtn'] == 'Not Available', 'processor_gnrtn'] = mode_value

In [141]:
with open('label_encoders_ms2.pkl', 'rb') as f:
    loaded_label_encoders = pickle.load(f)

d:\myApps\anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [142]:
def encoding_test(classes, X_test, column, label_encoders, cnt):
    encoded_values = []
    errors = {}
    for val in X_test[column]:
        try:
            if val in errors.keys():
                encoded_values.append(errors[val])
            else:
                val = label_encoders[column].transform(val)
                encoded_values.append(val)
        except ValueError as e:
            errors[val] = cnt
            encoded_values.append(cnt)
            np.append(label_encoders[column].classes_, cnt)
            cnt+=1
    return encoded_values

In [143]:

encode = ['brand','processor_brand','processor_name','ram_type','os','weight','warranty', 'processor_gnrtn', 'rating']

for column in encode:
    
    train_values_count = len(loaded_label_encoders[column].classes_)
    try:
        # Transform the test data
        data[column] = loaded_label_encoders[column].transform(data[column])
    except ValueError as e:
        # If ValueError occurs (unseen labels), handle it by assigning a default value
        print(f"Error occurred for column '{column}': {e}")
        data[column] = encoding_test(loaded_label_encoders[column].classes_, data, column, loaded_label_encoders, train_values_count)
        print(data[column].unique())
        #X_test[column] = X_test[column].apply(encoding_test(label_encoders[column].classes_))

In [144]:
def one_hot_encoding(df):
    one_encoded_data = pd.get_dummies(df, columns=df.columns)
    one_encoded_data [one_encoded_data  != False] = 1
    one_encoded_data [one_encoded_data  != True] = 0
    one_encoded_data.head()
    for col in one_encoded_data.columns:
        one_encoded_data[col] = one_encoded_data[col].astype(int)
    one_encoded_data.info()
    return one_encoded_data

In [145]:
def conc_datas(df1, df2):
    result = pd.concat([df1, df2], axis=1)
    return result

In [146]:
with open('selected_features_ms2.pkl', 'rb') as f:
    selector = pickle.load(f)

d:\myApps\anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SelectKBest from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [147]:
with open('ogreg_ms2.pkl', 'rb') as f:
    ogreg_ms2 = pickle.load(f)

d:\myApps\anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [148]:
with open('svc_classifier.pkl', 'rb') as f:
    svc_classifier = pickle.load(f)

d:\myApps\anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [149]:
with open('rf_classifier.pkl', 'rb') as f:
    rf_classifier = pickle.load(f)

d:\myApps\anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\myApps\anaconda\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [150]:
def normalization(df):
    df_normalized = np.log(df+1e-10)
    df_normalized[np.isinf(df_normalized)] = 0
    return df_normalized

In [151]:
def split_GB(df):    
    colToProcess=['ram_gb','ssd','hdd','graphic_card_gb']
    for col in colToProcess:
        splitted_data = df[col].str.split(' ')
        df[col] = [row[0] for row in splitted_data]
        df[col] = df[col].astype(int)
    
    return df

In [152]:
has_ssd = []
for index, row in data.iterrows():
    ssd = row['ssd']
    hdd = row['hdd']
    
    if ssd == '0 GB' and hdd != '0 GB':
        has_ssd.append('has only hdd')
    elif ssd != '0 GB' and hdd == '0 GB':
        has_ssd.append('has only ssd')
    else:
        has_ssd.append('has ssd and hdd')


In [153]:
data = split_GB(data)

In [154]:
x = data.drop('rating', axis=1)
y = data['rating']

In [155]:
x.head()
y.head()

0    0
1    1
2    0
3    1
4    0
Name: rating, dtype: int32

In [156]:
data_hot_encoded = one_hot_encoding(x[['Touchscreen','msoffice']])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723 entries, 0 to 722
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Touchscreen_No   723 non-null    int32
 1   Touchscreen_Yes  723 non-null    int32
 2   msoffice_No      723 non-null    int32
 3   msoffice_Yes     723 non-null    int32
dtypes: int32(4)
memory usage: 11.4 KB


In [157]:
all_data_encoded = conc_datas(x, data_hot_encoded)

In [158]:
all_data_encoded.drop(['Touchscreen', 'msoffice'], axis=1, inplace=True)
all_data_encoded

,brand,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,graphic_card_gb,weight,warranty,Price,Number of Ratings,Number of Reviews,Touchscreen_No,Touchscreen_Yes,msoffice_No,msoffice_Yes
0,1,1,6,1,4,5,0,512,4,0,0,3,45990,0,0,1,0,1,0
1,1,1,2,0,8,4,0,512,0,4,0,3,106167,1881,241,1,0,1,0
2,1,1,0,1,4,1,0,512,4,0,0,3,22990,0,0,1,0,1,0
3,5,1,1,1,4,1,256,0,4,0,0,0,44990,42,4,1,0,0,1
4,4,0,9,1,16,1,512,0,4,2,0,0,79990,23,6,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,5,1,1,1,4,1,256,0,4,0,0,3,39998,10,1,1,0,1,0
719,4,1,1,1,8,1,512,0,4,0,2,3,44990,0,0,1,0,1,0
720,5,1,2,1,8,1,512,0,4,0,0,2,64990,48,12,0,1,0,1
721,6,1,2,0,8,4,0,512,0,8,0,3,179990,24,5,1,0,1,0


In [159]:
selected_features = selector.transform(all_data_encoded)
normalized_data = normalization(selected_features)

d:\myApps\anaconda\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectKBest was fitted without feature names
  warnings.warn(


In [160]:
prediction = ogreg_ms2.predict(normalized_data)
print("test_logReg: ", metrics.accuracy_score(prediction, y))

test_logReg:  0.8990318118948825


In [161]:
prediction = svc_classifier.predict(normalized_data)
print("test_SVC: ",metrics.accuracy_score(prediction, y))

test_SVC:  0.8990318118948825


In [162]:
prediction = rf_classifier.predict(normalized_data)
print("test_RF: ",metrics.accuracy_score(prediction, y))

test_RF:  0.9612724757952974
